In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")
pinecone_api_key = os.getenv("PINECONE_API_KEY")

In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model='all-MiniLM-L6-v2')

d:\Learning\Codes\Agentic AI\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
google_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [4]:
google_embeddings.embed_query("Hello AI")

[0.01908188685774803,
 -0.04838583618402481,
 -0.0033970330841839314,
 -0.015432494692504406,
 0.026915451511740685,
 -0.037899527698755264,
 0.03683774173259735,
 0.0024586953222751617,
 0.031139591708779335,
 -0.002974053379148245,
 0.05297260358929634,
 0.027224618941545486,
 -0.03135930001735687,
 -0.03521350398659706,
 0.012696987949311733,
 -0.03398113697767258,
 -0.005759889259934425,
 0.04882349446415901,
 -0.025584226474165916,
 -0.016702925786376,
 0.03510289266705513,
 -0.0072209034115076065,
 -0.011071284301578999,
 -0.03638654947280884,
 0.02125084027647972,
 0.018635960295796394,
 0.022123489528894424,
 -0.053073909133672714,
 -0.03174823150038719,
 0.03385208174586296,
 -0.03267931193113327,
 0.05955952778458595,
 -0.04031715542078018,
 -0.01112906914204359,
 0.04654634743928909,
 -0.05826421454548836,
 0.03411427512764931,
 0.019681427627801895,
 0.008696145378053188,
 -0.004346166737377644,
 0.0029152834322303534,
 -0.08902806788682938,
 -0.015501082874834538,
 -0.0179

In [5]:
len(google_embeddings.embed_query("Hello AI"))

768

In [ ]:
from pinecone import Pinecone
pc=Pinecone()